In [1]:
import Pkg;
Pkg.activate(@__DIR__);
Pkg.instantiate()

  Activating environment at `~/Git/tinympc-julia/state_constraints/Project.toml`


In [2]:
using RobotZoo:Quadrotor
using RobotDynamics
using ForwardDiff
using TrajOptPlots
using BlockDiagonals
using LinearAlgebra
using StaticArrays
using SparseArrays

using MeshCat
using ColorTypes
using GeometryBasics: HyperRectangle, Cylinder, Vec, Point, Mesh
using CoordinateTransformations
using Rotations

using Plots
using Printf


include("../quaternion-stuff.jl")

E (generic function with 1 method)

In [3]:
# Quadrotor parameters
m = 0.035
ℓ = 0.046/sqrt(2)
J = [16.6e-6 0.83e-6 0.72e-6;
    0.83e-6 16.6e-6 1.8e-6;
    0.72e-6 1.8e-6 29.3e-6];
gravity = 9.81
# thrustToTorque = 0.005964552
thrustToTorque = 0.0008
scale = 65535
kt = 2.245365e-6*scale # u is PWM in range [0...1]
# kt=1
km = kt*thrustToTorque #4.4733e-8

# h = 1/500
h = 1/20

Nx = 13     # number of states (quaternion)
Nx̃ = 12     # number of states (linearized)
Nu = 4      # number of controls
nx = Nx̃
nu = Nu

#Goal state
@show uhover = (m*gravity/kt/4)*ones(4) # m = 30g and max thrust = 60g
# @show uhover = (m*gravity/4)*ones(4) # m = 30g and max thrust = 60g
rg = [0.0; 0; 0.0]
qg = [1.0; 0; 0; 0]
vg = zeros(3)
ωg = zeros(3)
xg = [rg; qg; vg; ωg];

uhover = (((m * gravity) / kt) / 4) * ones(4) = [0.5833333520642209, 0.5833333520642209, 0.5833333520642209, 0.5833333520642209]


In [4]:
function quad_dynamics(x,u)
  r = x[1:3]
  q = x[4:7]/norm(x[4:7]) #normalize q just to be careful
  v = x[8:10]
  ω = x[11:13]
  Q = qtoQ(q)
  
  ṙ = v
  q̇ = 0.5*L(q)*H*ω
  
  b = 1/m

  v̇ = [0; 0; -gravity] + (1/m)*Q*[zeros(2,4); kt*ones(1,4)]*u 

  ω̇ = J\(-hat(ω)*J*ω + [-ℓ*kt -ℓ*kt ℓ*kt ℓ*kt; -ℓ*kt ℓ*kt ℓ*kt -ℓ*kt; -km km -km km]*u)
  
  return [ṙ; q̇; v̇; ω̇]
end
function quad_dynamics_rk4(x,u)
  #RK4 integration with zero-order hold on u
  f1 = quad_dynamics(x, u)
  f2 = quad_dynamics(x + 0.5*h*f1, u)
  f3 = quad_dynamics(x + 0.5*h*f2, u)
  f4 = quad_dynamics(x + h*f3, u)
  xn = x + (h/6.0)*(f1 + 2*f2 + 2*f3 + f4)
  xn[4:7] = xn[4:7]/norm(xn[4:7]) #re-normalize quaternion
  return xn
end

quad_dynamics_rk4 (generic function with 1 method)

In [5]:
#Linearize dynamics about hover
Adyn = ForwardDiff.jacobian(x->quad_dynamics_rk4(x,uhover),xg)
Bdyn = ForwardDiff.jacobian(u->quad_dynamics_rk4(xg,u),uhover);
Ãdyn = Array(E(qg)'*Adyn*E(qg))
B̃dyn = Array(E(qg)'*Bdyn);

In [45]:
vis = Visualizer()
quad_model = Quadrotor()
TrajOptPlots.set_mesh!(vis, quad_model)

function TransformPlane(visObject, center, dir)
    # dir[1] = x, dir[2] = y (for three dimensions, take cross product of [1,0,0] and 3D dir vector to get axis of rotation)
    width = 2
    height = 1
    dir_inv = -[dir[2]; -dir[1]]/norm(dir)
    bl = [center[1:2] + dir_inv*width/2; 0] # bottom left corner
    settransform!(visObject, Translation(bl[1], bl[2], bl[3]) ∘ LinearMap(AngleAxis(-atan(dir[2], -dir[1]), 0, 0, 1)))
end

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8707
└ @ MeshCat /home/sam/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


TransformPlane (generic function with 1 method)

In [51]:
# Objective function

# Cost weights
max_dev_x = [0.1; 0.1; 0.1; 0.5; 0.5; 0.05;  0.5; 0.5; 0.5;  0.7; 0.7; 0.5]
max_dev_u = [0.5; 0.5; 0.5; 0.5]
Q = spdiagm(1 ./(max_dev_x.^2))
R = spdiagm(1 ./(max_dev_u.^2))
Qf = Q

# Penalty
ρ = 5
R̃ = R + ρ*I;

# Precompute
cache = (
    Ã = Ãdyn,
    B̃ = B̃dyn,
    Kinf = zeros(nu,nx),
    Pinf = zeros(nx,nx),
    Quu_inv = zeros(nu,nu),
    AmBKt = zeros(nx,nx), 
    coeff_d2p = zeros(nx,nu), 
)


Tfinal = 10.0
N = Int(Tfinal/h)+1

# Compute Kinf, Pinf
Nriccati = 10000
P = [zeros(nx,nx) for i = 1:Nriccati];   # cost to go quadratic term
K = [zeros(nu,nx) for i = 1:Nriccati-1]; # feedback gain
P[Nriccati] .= Qf;
for k = (N-1):-1:1
    K[k] = (R̃ + cache.B̃'*P[k+1]*cache.B̃)\(cache.B̃'*P[k+1]*cache.Ã);
    P[k] = Q + cache.Ã'*P[k+1]*(cache.Ã - cache.B̃*K[k]);
end

# Cache precomputed values
cache.Kinf .= K[1];
cache.Pinf .= P[1];
cache.Quu_inv .= (R̃ + cache.B̃'*cache.Pinf*cache.B̃)\I;
cache.AmBKt .= (cache.Ã - cache.B̃*cache.Kinf)';
cache.coeff_d2p .= cache.Kinf'*R̃ - cache.AmBKt*cache.Pinf*cache.B̃;



# Create reference sketch trajectory
X̃ref = [zeros(nx) for i = 1:N]
Xref = [[0; 0; .5; 1; zeros(9)] for i = 1:N]
Uref = [zeros(nu) for i = 1:N-1]



# Convert (13 state) Xref to reduced form (12 state) X̃ref
for k = 1:N
    x = Xref[k]
    qx = x[4:7]
    ϕ = qtorp(L(qg)'*qx)   
    X̃ref[k] = [x[1:3]; ϕ; x[8:10]; x[11:13]]
end

# Set initial state
x0 = X̃ref[1] + [0; 0; 0; zeros(9)]

# Visualize reference
delete!(vis["XrefLine"])
XrefLine = [Point(x_[1], x_[2], x_[3]) for x_ in Xref]
setobject!(vis["XrefLine"], Object(PointCloud(XrefLine), 
        LineBasicMaterial(color=Colors.RGBA(0.,0.45,1.)), "Line"))

;

In [67]:
include("../tinyMPC-ADMM-dt-state.jl")

speye(N) = spdiagm(ones(N))


# Create obstacle in MeshCat
# Cylinder constraint
cylinder_center = [0.0; 2.0]
cylinder_velocity = .2 * h

r_vis = 0.1
r_obstacle = 0.75
setobject!(vis["cylinder"], Cylinder(Point(cylinder_center[1], cylinder_center[2], 0.0), Point(cylinder_center[1], cylinder_center[2], 1.0), r_vis))
bounding_cylinder_material = MeshPhongMaterial(color=RGBA(1, 1, 1, 0.25))
setobject!(vis["bounding cylinder"], Cylinder(Point(cylinder_center[1], cylinder_center[2], -.05), Point(cylinder_center[1], cylinder_center[2], 1.05), r_obstacle), bounding_cylinder_material)



# MPC setup
Nh = 20

skip_vis = 1

# Create constraint planes
thin_rect_material = MeshPhongMaterial(color=RGBA(0, 1, 0, 0.25))
constraintPlanes = [setobject!(vis["linearized constraint " * string(k)], HyperRectangle(Vec(0.0,0.0,0.0), Vec(0.001,2.0,1.0)), thin_rect_material) for k in 1:skip_vis:Nh]

# Define number of state constraints (nm)
nm = 1

# Instantiate TinyMPC-ADMM variables

x = [zeros(nx) for i = 1:Nh]
u = [zeros(nu) for i = 1:Nh-1]

q = [zeros(nx) for i = 1:Nh];
r = [zeros(nu) for i = 1:Nh-1];

p = [zeros(nx) for i = 1:Nh];      # cost to go linear term
p[Nh] = q[Nh];

d = [zeros(nu) for i = 1:Nh-1];    # feedforward control

# Input constraints
umin = -[1.0; 1.0; 1.0; 1.0]*.5
umax = [1.0; 1.0; 1.0; 1.0]*.5

# State constraints
# xmin = [-Inf .* ones(nx) for i = 1:Nh]
# xmax = [[Inf; 0.5; Inf .* ones(10)] for i = 1:Nh]
xmin = [-Inf for i = 1:Nh] # Only one -Inf per knot point because only one inequality constraint (for now)
xmax = [zeros(1) for i = 1:Nh]

# Aineq = [speye(nx) for i = 1:Nh]
Aineq = [zeros(nx) for i = 1:Nh]

for k = 1:skip_vis:Nh
    xc = cylinder_center - X̃ref[k][1:2]
    a = xc/norm(xc)

    Aineq[k][1:2] = a'

    q_c = cylinder_center - r_obstacle*a
    b = a'*q_c
    xmax[k][1] = b
    TransformPlane(constraintPlanes[cld(k,skip_vis)], q_c, a)
end

# Simulate
Nsim = N - Nh
xhist = [zeros(nx+1) for _ = 1:Nsim] # nx+1 because not reduced form
xhist[1] = [x0[1:3]; rptoq(x0[4:6]); x0[7:12]]
uhist = [zeros(nu) for _ = 1:Nsim-1]


for k = 1:Nsim-1 # Nsim-1 knotpoints for control, Nsim knotpoints for state

    # State slack (auxiliary) and dual variables
    v = [zeros(nx) for i = 1:Nh]
    vnew = [zeros(nx) for i = 1:Nh]
    g = [zeros(nx) for i = 1:Nh]

    # Input slack (auxiliary) and dual variables
    z = [zeros(nu) for i = 1:Nh-1]
    znew = [zeros(nu) for i = 1:Nh-1]
    y = [zeros(nu) for i = 1:Nh-1]

    x[1] = [xhist[k][1:3]; qtorp(xhist[k][4:7]); xhist[k][8:end]]

    params = (
        N = Nh,
        Q = Q, R = R, Qf = Qf,
        A = Aineq,
        umin = umin, umax = umax,
        xmin = xmin, xmax = xmax,
        Xref = X̃ref[k:k+Nh-1], Uref = Uref[k:k+Nh-2], 
        cache = cache
    );

    # Solve
    Δu, status, iter = solve_admm!(vis, params, q, r, p, d, x,v,vnew,g, u,z,znew,y; ρ=ρ, abs_tol=1e-2, max_iter=100)
    # @show iter, status
    

    # Visualize solution x
    delete!(vis["xLinearized"])
    x_linearized = [Point(x_[1], x_[2], x_[3]+0.01) for x_ in x]
    setobject!(vis["xLinearized"], Object(PointCloud(x_linearized), 
    LineBasicMaterial(color=Colors.RGBA(1,0.6,0.0)), "Line"))

    # Roll out x_k+1 with actual dynamics
    uhist[k] = Δu[1] + uhover
    xhist[k+1] = quad_dynamics_rk4(xhist[k], uhist[k])
    
    # Roll out with real dynamics
    x_real = [zeros(nx+1) for _ = 1:Nh]
    x_real[1] = xhist[k]
    for j = 1:Nh-1
        x_real[j+1] = quad_dynamics_rk4(x_real[j], uhist[k])
    end

    # Create constraints for each knot point in the horizon
    #   assuming the cylinder moves at constant velocity between each step
    cylinder_to_quad = x_real[1][1:2] - cylinder_center
    cylinder_to_quad_dir = cylinder_to_quad / norm(cylinder_to_quad)
    cylinder_dist_traveled = cylinder_velocity * cylinder_to_quad_dir
    
    # Update linearized cylinder constraints
    for j = 1:skip_vis:Nh
        cylinder_center_prediction = cylinder_center + cylinder_dist_traveled*j
        xc = cylinder_center_prediction - x_real[j][1:2]
        a = xc/norm(xc) # hyperplane projection expects normalized a

        Aineq[j][1:2] = a'

        q_c = cylinder_center_prediction - r_obstacle*a
        b = a'*q_c
        xmax[j][1] = b
        TransformPlane(constraintPlanes[cld(j,skip_vis)], q_c, a)
    end

    # Update actual cylinder location at x=0
    cylinder_center += cylinder_dist_traveled
    
    setobject!(vis["cylinder"], Cylinder(Point(cylinder_center[1], cylinder_center[2], 0.0), Point(cylinder_center[1], cylinder_center[2], 1.0), r_vis))
    setobject!(vis["bounding cylinder"], Cylinder(Point(cylinder_center[1], cylinder_center[2], -.05), Point(cylinder_center[1], cylinder_center[2], 1.05), r_obstacle), bounding_cylinder_material)

end

delete!(vis["xHistline"])
xHistline = [Point(x_[1], x_[2], x_[3]) for x_ in xhist]
setobject!(vis["xHistline"], Object(PointCloud(xHistline), 
LineBasicMaterial(color=Colors.RGBA(0.75,0.5,0.8)), "Line"))

MeshCat Visualizer with path /meshcat/xHistline at http://127.0.0.1:8707

In [68]:
xhist

181-element Vector{Vector{Float64}}:
 [0.0, 0.0, 0.5, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [1.245208733440633e-19, 3.497595128247174e-19, 0.4999760000437232, 1.0, -8.57358773318221e-17, 3.052356242166947e-17, 5.92059601269174e-17, 9.961669867525064e-18, 2.7980761025977393e-17, -0.000959998251071692, -6.858870186545766e-15, 2.4418849937335576e-15, 4.7364768101533914e-15]
 [1.9843741633207617e-18, 5.130834838020799e-18, 0.4999282067399356, 1.0, -2.2724464864409485e-16, 1.19484140176275e-16, 2.0754482023302348e-16, 7.897776126155674e-17, 1.8639932591745732e-16, -0.000951733900432572, -4.461831518436053e-15, 4.6749612266348846e-15, 7.130631998335095e-15]
 [9.341311710571555e-18, 2.0852816003452196e-17, 0.49988887559770495, 1.0, -3.1741939801092056e-16, 1.1891789407574305e-16, 4.0493108154672994e-16, 2.1520709706204368e-16, 4.572334181199921e-16, -0.0006215117887927058, -2.752148430910007e-15, -4.720260914677441e-15, 8.660268906761421e-15]
 [2.245384033748787e-17, 5.1952591141

In [69]:
X1 = [SVector{13}(x_) for x_ in xhist];
visualize!(vis, quad_model, Tfinal, X1)

In [28]:
# constraintPlanes = [setobject!(vis["linearized constraint " * string(k)], HyperRectangle(Vec(0.0,0.0,0.0), Vec(0.001,2.0,1.0)), thin_rect_material) for k in 1:Nh]
for k in 1:N
    delete!(vis["linearized constraint " * string(k)])
end